In [1]:
#!usr/bin/env python
#-*- coding: utf-8 -*-

import sys
import os
import time
from sklearn import metrics
import numpy as np
import cPickle as pickle
import logging

reload(sys)
sys.setdefaultencoding('utf8')

# Multinomial Naive Bayes Classifier
def naive_bayes_classifier(train_x, train_y):
    from sklearn.naive_bayes import MultinomialNB
    model = MultinomialNB(alpha=0.01)
    model.fit(train_x, train_y)
    return model


# KNN Classifier
def knn_classifier(train_x, train_y):
    from sklearn.neighbors import KNeighborsClassifier
    model = KNeighborsClassifier()
    model.fit(train_x, train_y)
    return model


# Logistic Regression Classifier
def logistic_regression_classifier(train_x, train_y):
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(penalty='l2')
    model.fit(train_x, train_y)
    return model


# Random Forest Classifier
def random_forest_classifier(train_x, train_y):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=8)
    model.fit(train_x, train_y)
    return model


# Decision Tree Classifier
def decision_tree_classifier(train_x, train_y):
    from sklearn import tree
    model = tree.DecisionTreeClassifier()
    model.fit(train_x, train_y)
    return model


# GBDT(Gradient Boosting Decision Tree) Classifier
def gradient_boosting_classifier(train_x, train_y):
    from sklearn.ensemble import GradientBoostingClassifier
    model = GradientBoostingClassifier(n_estimators=200)
    model.fit(train_x, train_y)
    return model


# SVM Classifier
def svm_classifier(train_x, train_y):
    from sklearn.svm import SVC
    model = SVC(kernel='rbf', probability=True)
    model.fit(train_x, train_y)
    return model

# SVM Classifier using cross validation
def svm_cross_validation(train_x, train_y):
    from sklearn.grid_search import GridSearchCV
    from sklearn.svm import SVC
    model = SVC(kernel='rbf', probability=True)
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}
    grid_search = GridSearchCV(model, param_grid, n_jobs = 1, verbose=1)
    grid_search.fit(train_x, train_y)
    best_parameters = grid_search.best_estimator_.get_params()
    for para, val in best_parameters.items():
        print para, val
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)
    model.fit(train_x, train_y)
    return model

def read_data(data_file):
    import gzip
    f = gzip.open(data_file, "rb")
    train, val, test = pickle.load(f)
    f.close()
    train_x = train[0]
    train_y = train[1]
    test_x = test[0]
    test_y = test[1]
    return train_x, train_y, test_x, test_y

logging.basicConfig(level=logging.DEBUG,
                format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                datefmt='%a, %d %b %Y %H:%M:%S',
                filename='myapp.log',
                filemode='w')

#打印日志
#定义一个StreamHandler，将INFO级别或更高的日志信息打印到标准错误，并将其添加到当前的日志处理对象#
console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

#logging.debug('This is debug message')
#logging.info('This is info message')
#logging.warning('This is warning message')

    
if __name__ == '__main__':
    data_file = "mnist.pkl.gz"
    thresh = 0.5
    model_save_file = None
    model_save = {}
    
    test_classifiers = ['NB', 'KNN', 'LR', 'RF', 'DT', 'SVM', 'GBDT']
    classifiers = {'NB':naive_bayes_classifier, 
                  'KNN':knn_classifier,
                   'LR':logistic_regression_classifier,
                   'RF':random_forest_classifier,
                   'DT':decision_tree_classifier,
                  'SVM':svm_classifier,
                'SVMCV':svm_cross_validation,
                 'GBDT':gradient_boosting_classifier
    }
    
    #print 'reading training and testing data...'
    logging.info('reading training and testing data...')
    train_x, train_y, test_x, test_y = read_data(data_file)
    num_train, num_feat = train_x.shape
    num_test, num_feat = test_x.shape
    is_binary_class = (len(np.unique(train_y)) == 2)
    #print '******************** Data Info *********************'
    #print '#training data: %d, #testing_data: %d, dimension: %d' % (num_train, num_test, num_feat)
    
    logging.info('******************** Data Info *********************')
    logging.info('#training data: %d, #testing_data: %d, dimension: %d' % (num_train, num_test, num_feat))
    for classifier in test_classifiers:
        #print '******************* %s ********************' % classifier
        logging.info('******************* %s ********************' % classifier)
        start_time = time.time()
        model = classifiers[classifier](train_x, train_y)
        #print 'training took %fs!' % (time.time() - start_time)
        logging.info('training took %fs!' % (time.time() - start_time))
        predict = model.predict(test_x)
        if model_save_file != None:
            model_save[classifier] = model
        if is_binary_class:
            precision = metrics.precision_score(test_y, predict)
            recall = metrics.recall_score(test_y, predict)
            #print 'precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall)
            logging.info('precision: %.2f%%, recall: %.2f%%' % (100 * precision, 100 * recall))
        accuracy = metrics.accuracy_score(test_y, predict)
        #print 'accuracy: %.2f%%' % (100 * accuracy)
        logging.info('accuracy: %.2f%%' % (100 * accuracy))

    if model_save_file != None:
        pickle.dump(model_save, open(model_save_file, 'wb'))


代码运行结果：
Mon, 04 Dec 2017 11:32:19 <ipython-input-1-6a108ab48155>[line:133] INFO reading training and testing data...
Mon, 04 Dec 2017 11:32:20 <ipython-input-1-6a108ab48155>[line:141] INFO ******************** Data Info *********************
Mon, 04 Dec 2017 11:32:20 <ipython-input-1-6a108ab48155>[line:142] INFO #training data: 50000, #testing_data: 10000, dimension: 784
Mon, 04 Dec 2017 11:32:20 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* NB ********************
Mon, 04 Dec 2017 11:32:20 <ipython-input-1-6a108ab48155>[line:149] INFO training took 0.291610s!
Mon, 04 Dec 2017 11:32:20 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 83.69%
Mon, 04 Dec 2017 11:32:20 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* KNN ********************
Mon, 04 Dec 2017 11:32:27 <ipython-input-1-6a108ab48155>[line:149] INFO training took 7.104816s!
Mon, 04 Dec 2017 11:44:59 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 96.64%
Mon, 04 Dec 2017 11:44:59 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* LR ********************
Mon, 04 Dec 2017 11:46:25 <ipython-input-1-6a108ab48155>[line:149] INFO training took 86.412181s!
Mon, 04 Dec 2017 11:46:25 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 91.99%
Mon, 04 Dec 2017 11:46:25 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* RF ********************
Mon, 04 Dec 2017 11:46:28 <ipython-input-1-6a108ab48155>[line:149] INFO training took 2.993257s!
Mon, 04 Dec 2017 11:46:28 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 93.74%
Mon, 04 Dec 2017 11:46:28 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* DT ********************
Mon, 04 Dec 2017 11:46:44 <ipython-input-1-6a108ab48155>[line:149] INFO training took 15.655863s!
Mon, 04 Dec 2017 11:46:44 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 87.33%
Mon, 04 Dec 2017 11:46:44 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* SVM ********************
Mon, 04 Dec 2017 12:37:13 <ipython-input-1-6a108ab48155>[line:149] INFO training took 3029.203229s!
Mon, 04 Dec 2017 12:40:23 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 94.35%
Mon, 04 Dec 2017 12:40:23 <ipython-input-1-6a108ab48155>[line:145] INFO ******************* GBDT ********************
Mon, 04 Dec 2017 14:00:22 <ipython-input-1-6a108ab48155>[line:149] INFO training took 4799.003342s!
Mon, 04 Dec 2017 14:00:23 <ipython-input-1-6a108ab48155>[line:160] INFO accuracy: 96.17%

在这个数据集中，由于数据分布的团簇性较好为，因此KNN的效果不错。GBDT是个非常不错的算法，在kaggle等数据竞赛中应用较多，且效果很好，SVM也具有不错的效果，但计算量较多，训练时间较长。